<a href="https://colab.research.google.com/github/chinmay5/NLP-Praktikum/blob/master/Siamese_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [0]:
#!pip install torch==1.0.1 -f https://download.pytorch.org/whl/cu100/stable # CUDA 10.0 build

In [5]:
!pip3 install torchvision
!pip install nltk
!pip install tqdm

In [0]:
#coding=utf-8
import re
import nltk
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
stops1 = set(stopwords.words("spanish"))

def clean_sent(sent):
    sent = sent.lower()
    sent = re.sub(u'[_"\-;%()|+&=*%.,!?:#$@\[\]/]',' ',sent)
    sent = re.sub('¡',' ',sent)
    sent = re.sub('¿',' ',sent)
    sent = re.sub('Á','á',sent)
    sent = re.sub('Ó','ó',sent)
    sent = re.sub('Ú','ú',sent)
    sent = re.sub('É','é',sent)
    sent = re.sub('Í','í',sent)
    return sent
  
def cleanSpanish(df):
    df['spanish1'] = df.spanish1.map(lambda x: ' '.join([ word for word in
                                                         nltk.word_tokenize(clean_sent(x))]))
    df['spanish2'] = df.spanish2.map(lambda x: ' '.join([ word for word in
                                                         nltk.word_tokenize(clean_sent(x))]))
    
def removeSpanishStopWords(df, stop):
	df['spanish1'] = df.spanish1.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x)
                                                         if word not in stop]))
	df['spanish2'] = df.spanish2.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x)
                                                         if word not in stop]))


def data_preprocessing():

    # Training data
    import os
    os.chdir("/content/")
    !ls

    df_train_en_sp = pd.read_csv('./cikm_english_train_20180516.txt', sep='	', header=None,
                                 error_bad_lines=False)
    df_train_sp_en = pd.read_csv('./cikm_spanish_train_20180516.txt', sep='	', header=None,
                                 error_bad_lines=False)
    df_train_en_sp.columns = ['english1', 'spanish1', 'english2', 'spanish2', 'result']
    df_train_sp_en.columns = ['spanish1', 'english1', 'spanish2', 'english2', 'result']
    train1 = pd.DataFrame(pd.concat([df_train_en_sp['spanish1'], df_train_sp_en['spanish1']], axis=0))
    train2 = pd.DataFrame(pd.concat([df_train_en_sp['spanish2'], df_train_sp_en['spanish2']], axis=0))
    train_data = pd.concat([train1, train2], axis=1).reset_index()
    train_data = train_data.drop(['index'], axis=1)
    result = pd.DataFrame(pd.concat([df_train_en_sp['result'], df_train_sp_en['result']], axis=0)).reset_index()
    result = result.drop(['index'], axis=1)
    # pd.get_dummies(result['result']).head()
    train_data['result'] = result

    # Evaluation data
    test_data = pd.read_csv('./cikm_test_a_20180516.txt', sep='	', header=None, error_bad_lines=False)
    test_data.columns = ['spanish1', 'spanish2']


    cleanSpanish(train_data)
    removeSpanishStopWords(train_data, stops1)
    cleanSpanish(test_data)
    removeSpanishStopWords(test_data, stops1)

    train_data.replace('', np.nan, inplace=True)
    dirty_data = train_data[train_data.isnull().any(axis=1)]
    print ('dirty sample count:', dirty_data.shape[0])
    print ('positive dirty training sample:', len(dirty_data[dirty_data['result'] == 1]))
    print ('negative dirty training sample:', len(dirty_data[dirty_data['result'] == 0]))

    train_data = train_data.dropna()
    test_data.replace('', np.nan, inplace=True)
    test_data = test_data.dropna()
    print ('Train sample count:', train_data.shape[0], 'Test sample count:', test_data.shape[0])

    train_data.columns = ['s1', 's2', 'label']
    test_data.columns = ['s1', 's2']

    train_data.to_csv("cleaned_train.csv", index=False)
    test_data.to_csv("cleaned_test.csv", index=False)

def get_embedding(word_dict, embedding_path, embedding_dim=300):
    # find existing word embeddings
    word_vec = {}
    with open(embedding_path) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word in word_dict:
                word_vec[word] = np.array(list(map(float, vec.split())))
    print('Found {0}/{1} words with embedding vectors'.format(
        len(word_vec), len(word_dict)))
    missing_word_num = len(word_dict) - len(word_vec)
    missing_ratio = round(float(missing_word_num) / len(word_dict), 4) * 100
    print('Missing Ratio: {}%'.format(missing_ratio))

    # handling unknown embeddings
    for word in word_dict:
        if word not in word_vec:
            # If word not in word_vec, create a random embedding for it
            new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
            word_vec[word] = new_embedding
    print ("Filled missing words' embeddings.")
    print ("Embedding Matrix Size: ", len(word_vec))

    return word_vec

def save_embed(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    print ('Embedding saved')

def load_embed(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [0]:
"""
  Some of the variables that are going to be treated as constants and used throughout the code
"""

data_preprocess = True # Would preprocess the data and generate the embeddings
make_dict = True #  Would generate the embeddings

In [8]:
""" Data Preprocessing """

import yaml
import os
os.chdir("/content/")
!ls

with open('siamese-config.yaml') as f:
  config = yaml.load(f)

  
if data_preprocess:
    print ('Pre-processing Original Data ...')
    data_preprocessing()
    print ('Data Pre-processing Done!')

cikm_english_train_20180516.txt  cleaned_train.csv    wiki.en.vec
cikm_spanish_train_20180516.txt  embedding.pkl	      wiki.es.vec
cikm_test_a_20180516.txt	 sample_data
cleaned_test.csv		 siamese-config.yaml
Pre-processing Original Data ...
cikm_english_train_20180516.txt  cleaned_train.csv    wiki.en.vec
cikm_spanish_train_20180516.txt  embedding.pkl	      wiki.es.vec
cikm_test_a_20180516.txt	 sample_data
cleaned_test.csv		 siamese-config.yaml
dirty sample count: 73
positive dirty training sample: 5
negative dirty training sample: 68
Train sample count: 21327 Test sample count: 4998
Data Pre-processing Done!


In [0]:
from torch import nn
from torch.autograd import Variable
import torch



class LSTMEncoder(nn.Module):
    def __init__(self, config):
        super(LSTMEncoder, self).__init__()
        self.embed_size = config['model']['embed_size']
        self.batch_size = config['model']['batch_size']
        self.hidden_size = config['model']['encoder']['hidden_size']
        self.num_layers = config['model']['encoder']['num_layers']
        self.bidir = config['model']['encoder']['bidirectional']
        if self.bidir:
            self.direction = 2
        else: self.direction = 1
        self.dropout = config['model']['encoder']['dropout']

        self.embedding = config['embedding_matrix']
        self.lstm = nn.LSTM(input_size=self.embed_size, hidden_size=self.hidden_size, dropout=self.dropout,
                            num_layers=self.num_layers, bidirectional=self.bidir)
        self.lstm = self.lstm.cuda()

    def initHiddenCell(self):
        rand_hidden = Variable(torch.randn(self.direction * self.num_layers, self.batch_size, self.hidden_size))
        rand_cell = Variable(torch.randn(self.direction * self.num_layers, self.batch_size, self.hidden_size))
        rand_hidden = rand_hidden.cuda()
        rand_cell = rand_cell.cuda()
        return rand_hidden, rand_cell

    def forward(self, input, hidden, cell):
        input = self.embedding(input).view(1, 1, -1)
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        return output, hidden, cell

In [0]:
class Siamese_lstm(nn.Module):
    def __init__(self, config):
        super(Siamese_lstm, self).__init__()

        self.encoder = LSTMEncoder(config)
        self.fc_dim = config['model']['fc_dim']

        self.input_dim = 5 * self.encoder.direction * self.encoder.hidden_size
        # self.classifier = nn.Sequential(
        #     nn.Linear(self.input_dim, self.fc_dim),
        #     nn.Linear(self.fc_dim, 2)
        # )
        self.classifier = nn.Sequential(
            nn.Linear(self.input_dim, int(self.input_dim/2)),
            nn.Linear(int(self.input_dim/2), 2)
        ).cuda()

    def forward(self, s1, s2):

        # init hidden, cell
        h1, c1 = self.encoder.initHiddenCell()
        h2, c2 = self.encoder.initHiddenCell()

        # input one by one

        for i in range(len(s1)):

            v1, h1, c1 = self.encoder(s1[i], h1, c1)
            
        for j in range(len(s2)):
            v2, h2, c2 = self.encoder(s2[j], h2, c2)
            
        # utilize these two encoded vectors
        features = torch.cat((v1,torch.abs(v1 - v2),v2,v1*v2, (v1+v2)/2), 2)
        # features = v1-v2
        output = self.classifier(features)

        return output

In [11]:
!cat siamese-config.yaml

experiment_name: 'siamese-baseline'

task: 'train'
make_dict: False
data_preprocessing: False

ckpt_dir: 'ckpt/'

training:
    num_epochs: 20
    learning_rate: 0.01
    # options = ['adam', 'adadelta', 'rmsprop']
    optimizer: 'sgd'


embedding:
    full_embedding_path: 'input/wiki.es.vec'
    cur_embedding_path: 'input/embedding.pkl'

model:
    fc_dim: 100
    name: 'siamese'
    embed_size: 300
    batch_size: 1
    embedding_freeze: False
    encoder:
        hidden_size: 150
        num_layers: 1
        bidirectional: False
        dropout: 0.0

result:
    filename: 'result.txt'
    filepath: 'res/'









In [0]:
from torch.utils.data import Dataset
from collections import Counter


class myDS(Dataset):

    def __init__(self, df, all_sents):
        # Assign vocabularies.
        self.s1 = df['s1'].tolist()
        self.s2 = df['s2'].tolist()
        self.label = df['label'].tolist()
        self.vocab = Vocab(all_sents, sos_token='<sos>', eos_token='<eos>', unk_token='<unk>')

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        # Split sentence into words.
        s1_words = self.s1[idx].split()
        s2_words = self.s2[idx].split()

        # Add <SOS> and <EOS> tokens.
        s1_words = [self.vocab.sos_token] + s1_words + [self.vocab.eos_token]
        s2_words = [self.vocab.sos_token] + s2_words + [self.vocab.eos_token]

        # Lookup word ids in vocabularies.
        s1_ids = [self.vocab.word2id(word) for word in s1_words]
        s2_ids = [self.vocab.word2id(word) for word in s2_words]
        label = self.label[idx]

        return s1_ids, s2_ids, label


In [0]:
class mytestDS(Dataset):

    def __init__(self, df, all_sents):
        # Assign vocabularies.
        self.s1 = df['s1'].tolist()
        self.s2 = df['s2'].tolist()
        self.vocab = Vocab(all_sents, sos_token='<sos>', eos_token='<eos>', unk_token='<unk>')

    def __len__(self):
        return len(self.s1)

    def __getitem__(self, idx):
        # Split sentence into words.
        s1_words = self.s1[idx].split()
        s2_words = self.s2[idx].split()

        # Add <SOS> and <EOS> tokens.
        s1_words = [self.vocab.sos_token] + s1_words + [self.vocab.eos_token]
        s2_words = [self.vocab.sos_token] + s2_words + [self.vocab.eos_token]

        # Lookup word ids in vocabularies.
        s1_ids = [self.vocab.word2id(word) for word in s1_words]
        s2_ids = [self.vocab.word2id(word) for word in s2_words]
        
        return s1_ids, s2_ids


In [0]:
class Vocab(object):
    def __init__(self, all_sents, max_size=None, sos_token=None, eos_token=None, unk_token=None):
        """Initialize the vocabulary.
        Args:
            iter: An iterable which produces sequences of tokens used to update
                the vocabulary.
            max_size: (Optional) Maximum number of tokens in the vocabulary.
            sos_token: (Optional) Token denoting the start of a sequence.
            eos_token: (Optional) Token denoting the end of a sequence.
            unk_token: (Optional) Token denoting an unknown element in a
                sequence.
        """
        self.max_size = max_size
        self.pad_token = '<pad>'
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.unk_token = unk_token

        # Add special tokens.
        id2word = [self.pad_token]
        if sos_token is not None:
            id2word.append(self.sos_token)
        if eos_token is not None:
            id2word.append(self.eos_token)
        if unk_token is not None:
            id2word.append(self.unk_token)

        # Update counter with token counts.
        counter = Counter()
        for x in all_sents:
            counter.update(x.split())

        # Extract lookup tables.
        if max_size is not None:
            counts = counter.most_common(max_size)
        else:
            counts = counter.items()
            counts = sorted(counts, key=lambda x: x[1], reverse=True)
        words = [x[0] for x in counts]
        id2word.extend(words)
        word2id = {x: i for i, x in enumerate(id2word)}

        self._id2word = id2word
        self._word2id = word2id

    def __len__(self):
        return len(self._id2word)

    def word2id(self, word):
        """Map a word in the vocabulary to its unique integer id.
        Args:
            word: Word to lookup.
        Returns:
            id: The integer id of the word being looked up.
        """
        if word in self._word2id:
            return self._word2id[word]
        elif self.unk_token is not None:
            return self._word2id[self.unk_token]
        else:
            raise KeyError('Word "%s" not in vocabulary.' % word)

    def id2word(self, id):
        """Map an integer id to its corresponding word in the vocabulary.
        Args:
            id: Integer id of the word being looked up.
        Returns:
            word: The corresponding word.
        """
        return self._id2word[id]

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
import pandas as pd
import numpy as np
""" Read Data """

train_data = pd.read_csv('cleaned_train.csv')
test_data = pd.read_csv('cleaned_test.csv')

# split dataset
msk = np.random.rand(len(train_data)) < 0.8
train = train_data[msk]
valid = train_data[~msk]
all_sents = train_data['s1'].tolist() + train_data['s2'].tolist() + test_data['s1'].tolist() + test_data['s2'].tolist()

# dataset
trainDS = myDS(train, all_sents)
validDS = myDS(valid, all_sents)

print ('Data size:',train_data.shape[0], test_data.shape[0])

Data size: 21327 4998


In [0]:
""" Get Embedding """

#full_embed_path = config['embedding']['full_embedding_path']
#cur_embed_path = config['embedding']['cur_embedding_path']

# TODO: Make sure we read it from the config file in the future
full_embed_path = "wiki.es.vec"
cur_embed_path = "embedding.pkl"

In [0]:
# Run this line if the embeddings need to be loaded


#!curl https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec > ./wiki.en.vec
#!curl https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec > ./wiki.es.vec

In [19]:
import os

if os.path.exists(cur_embed_path) and not make_dict:
    embed_dict = load_embed(cur_embed_path)
    print ('Loaded existing embedding.')
else:
    print ('Making embedding...')
    embed_dict = get_embedding(trainDS.vocab._id2word, full_embed_path)
    save_embed(embed_dict,"embedding.pkl")
    print ('Saved generated embedding.')

Making embedding...
Found 5141/5773 words with embedding vectors
Missing Ratio: 10.95%
Filled missing words' embeddings.
Embedding Matrix Size:  5773
Embedding saved
Saved generated embedding.


In [0]:
vocab_size = len(embed_dict)
# initialize nn embedding
embedding = nn.Embedding(vocab_size, config['model']['embed_size'])
embed_list = []
for word in trainDS.vocab._id2word:
    embed_list.append(embed_dict[word])
weight_matrix = np.array(embed_list)
# pass weights to nn embedding
embedding.weight = nn.Parameter(torch.from_numpy(weight_matrix).type(torch.FloatTensor), requires_grad = False)


In [0]:
# embedding
config['embedding_matrix'] = embedding
config['vocab_size'] = len(embed_dict)

In [23]:
# model
siamese = Siamese_lstm(config)
siamese.cuda()
siamese.encoder.cuda()


LSTMEncoder(
  (embedding): Embedding(5773, 300)
  (lstm): LSTM(300, 150)
)

In [0]:
vocab_size = len(embed_dict)
# initialize nn embedding
embedding = nn.Embedding(vocab_size, config['model']['embed_size'])
embed_list = []
for word in trainDS.vocab._id2word:
    embed_list.append(embed_dict[word])
weight_matrix = np.array(embed_list)
# pass weights to nn embedding
embedding.weight = nn.Parameter(torch.from_numpy(weight_matrix).type(torch.FloatTensor), requires_grad = False)

Code for preprocessing the text and other cleaning

Setting up of the Parameters

In [0]:
# optimizer
learning_rate = config['training']['learning_rate']
if config['training']['optimizer'] == 'sgd':
    optimizer = torch.optim.SGD(filter(lambda x: x.requires_grad, siamese.parameters()), lr=learning_rate)
elif config['training']['optimizer'] == 'adam':
    optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, siamese.parameters()), lr=learning_rate)
elif config['training']['optimizer'] == 'adadelta':
    optimizer = torch.optim.Adadelta(filter(lambda x: x.requires_grad, siamese.parameters()), lr=learning_rate)
elif config['training']['optimizer'] == 'rmsprop':
    optimizer = torch.optim.RMSprop(filter(lambda x: x.requires_grad, siamese.parameters()), lr=learning_rate)

In [0]:
# loss func
loss_weights = Variable(torch.FloatTensor([1, 3]))
if torch.cuda.is_available():
    loss_weights = loss_weights.cuda()
criterion = torch.nn.CrossEntropyLoss(loss_weights)

In [27]:
# Restore saved model (if one exists).
ckpt_path = os.path.join('./', config['experiment_name']+'.pt')

if os.path.exists(ckpt_path):
    print('Loading checkpoint: %s' % ckpt_path)
    ckpt = torch.load(ckpt_path)
    epoch = ckpt['epoch']
    siamese.load_state_dict(ckpt['siamese'])
    optimizer.load_state_dict(ckpt['optimizer'])
else:
    epoch = 0
    print ('Fresh start!')

Fresh start!


In [0]:
# log info
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
valid_log_string = '%s :: Epoch %i :: valid loss: %0.4f\n'


## Training Procedure

In [0]:
from torch.utils.data import DataLoader
from datetime import datetime
# save every epoch for visualization
train_loss_record = []
valid_loss_record = []
best_record = 10.0

# training
print ('Experiment:{}\n'.format(config['experiment_name']))

    
while (epoch < config['training']['num_epochs']):

    print ('Start Epoch{} Training...'.format(epoch))

    # loss
    train_loss = []
    train_loss_sum = []
    # dataloader
    train_dataloader = DataLoader(dataset=trainDS, shuffle=True, num_workers=2, batch_size=1)

    for idx, data in enumerate(train_dataloader, 0):

        # get data
        s1, s2, label = data
        
        # putting the data into cuda
        s1 = torch.from_numpy(np.array(s1))
        s2 = torch.from_numpy(np.array(s2))
        label = torch.from_numpy(np.array(label))
        
        s1 = s1.cuda()
        s2 = s2.cuda()
        label = label.cuda()
        
        # clear gradients
        optimizer.zero_grad()
        
        
        # input
        output = siamese(s1, s2)
        output = output.squeeze(0)

        # loss backward
        loss = criterion(output, Variable(label))
        loss.backward()
        optimizer.step()
        train_loss.append(loss.data.cpu())
        train_loss_sum.append(loss.data.cpu())

        # Every once and a while check on the loss
        if ((idx + 1) % 5000) == 0:
            print(train_log_string % (datetime.now(), epoch, idx + 1, len(train), np.mean(train_loss)))
            train_loss = []

    # Record at every epoch
    print ('Train Loss at epoch{}: {}\n'.format(epoch, np.mean(train_loss_sum)))
    train_loss_record.append(np.mean(train_loss_sum))

    # Valid
    print ('Epoch{} Validating...'.format(epoch))

    # loss
    valid_loss = []
    # dataloader
    valid_dataloader = DataLoader(dataset=validDS, shuffle=True, num_workers=2, batch_size=1)

    for idx, data in enumerate(valid_dataloader, 0):
        # get data
        s1, s2, label = data

        # putting the data into cuda
        s1 = torch.from_numpy(np.array(s1))
        s2 = torch.from_numpy(np.array(s2))
        label = torch.from_numpy(np.array(label))
        
        s1 = s1.cuda()
        s2 = s2.cuda()
        label = label.cuda()
        
        # input
        output = siamese(s1, s2)
        output = output.squeeze(0)

        # loss
        loss = criterion(output, Variable(label))
        valid_loss.append(loss.data.cpu())

    print(valid_log_string % (datetime.now(), epoch, np.mean(valid_loss)))
    # Record
    valid_loss_record.append(np.mean(valid_loss))

    epoch += 1

    # Keep track of best record
    if np.mean(valid_loss) < best_record:
        best_record = np.mean(valid_loss)
        # save the best model
        state_dict = {
            'epoch': epoch,
            'siamese': siamese.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(state_dict, ckpt_path)
        print ('Model saved!\n')

Experiment:siamese-baseline

Start Epoch3 Training...
2019-05-21 22:27:26.995351 :: Epoch 3 :: Iter 5000 / 17013 :: train loss: 0.3915
2019-05-21 22:28:47.501385 :: Epoch 3 :: Iter 10000 / 17013 :: train loss: 0.4048
2019-05-21 22:30:11.273321 :: Epoch 3 :: Iter 15000 / 17013 :: train loss: 0.3719
Train Loss at epoch3: 0.3892628252506256

Epoch3 Validating...
2019-05-21 22:31:30.875104 :: Epoch 3 :: valid loss: 0.3873

Model saved!

Start Epoch4 Training...
2019-05-21 22:32:52.124570 :: Epoch 4 :: Iter 5000 / 17013 :: train loss: 0.3404
2019-05-21 22:34:13.403940 :: Epoch 4 :: Iter 10000 / 17013 :: train loss: 0.3437
2019-05-21 22:35:35.186432 :: Epoch 4 :: Iter 15000 / 17013 :: train loss: 0.3472
Train Loss at epoch4: 0.34338438510894775

Epoch4 Validating...
2019-05-21 22:36:54.765014 :: Epoch 4 :: valid loss: 0.3658

Model saved!

Start Epoch5 Training...
2019-05-21 22:38:15.622470 :: Epoch 5 :: Iter 5000 / 17013 :: train loss: 0.3026
2019-05-21 22:39:37.149971 :: Epoch 5 :: Iter 10

Include Torch vision in here

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(train_loss_record)
plt.plot(valid_loss_record)

In [0]:

""" Inference """
# if config['taks'] == 'inference':
testDS = mytestDS(test_data, all_sents)
# Do not shuffle here
test_dataloader = DataLoader(dataset=testDS, num_workers=2, batch_size=1)

In [0]:
result = []
for idx, data in enumerate(test_dataloader, 0):

    # get data
    s1, s2 = data

    # input
    output = siamese(s1,s2)
    output = output.squeeze(0)

    # feed output into softmax to get prob prediction
    sm = nn.Softmax(dim=1)
    res = sm(output.data)[:,1]
    result += res.data.tolist()

result = pd.DataFrame(result)
print 'Inference Done.'

In [0]:
plt.show()

In [0]:
res_path = os.path.join(config['result']['filepath'], config['result']['filename'])
result.to_csv(res_path,header=False,index=False)

##Extra Section that will come in handy later

In [0]:
class ContrastiveLoss(nn.Module):
   
    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(1)  # squared distances
        losses = 0.5 * (target.float() * distances +
                        (1 + -1 * target).float() * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2)) #ReLU function does the same task as selecting max
        if size_average:
          return losses.mean() 
        return losses.sum()


class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        if size_average:
          return losses.mean()
        return losses.sum()


In [0]:
class OnlineContrastiveLoss(nn.Module):
    """
    The pair selector and embeddings are sent
    """

    def __init__(self, margin, pair_selector):
        super(OnlineContrastiveLoss, self).__init__()
        self.margin = margin
        self.pair_selector = pair_selector

    def forward(self, embeddings, target):
        positive_pairs, negative_pairs = self.pair_selector.get_pairs(embeddings, target)
        if embeddings.is_cuda:
            positive_pairs = positive_pairs.cuda()
            negative_pairs = negative_pairs.cuda()
        positive_loss = (embeddings[positive_pairs[:, 0]] - embeddings[positive_pairs[:, 1]]).pow(2).sum(1)
        negative_loss = F.relu(
            self.margin - (embeddings[negative_pairs[:, 0]] - embeddings[negative_pairs[:, 1]]).pow(2).sum(
                1).sqrt()).pow(2)
        loss = torch.cat([positive_loss, negative_loss], dim=0) #dim 0 is the rows
        return loss.mean()


class OnlineTripletLoss(nn.Module):
    """
    Triplet selector and the embeddings are sent
    """

    def __init__(self, margin, triplet_selector):
        super(OnlineTripletLoss, self).__init__()
        self.margin = margin
        self.triplet_selector = triplet_selector

    def forward(self, embeddings, target):

        triplets = self.triplet_selector.get_triplets(embeddings, target)

        if embeddings.is_cuda:
            triplets = triplets.cuda()

        ap_distances = (embeddings[triplets[:, 0]] - embeddings[triplets[:, 1]]).pow(2).sum(1)  # .pow(.5)
        an_distances = (embeddings[triplets[:, 0]] - embeddings[triplets[:, 2]]).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(ap_distances - an_distances + self.margin)

        return losses.mean(), len(triplets)


In [0]:
class TripletDataSet(Dataset):
    """
    Train: For each sample (anchor) randomly chooses a positive and negative samples
    Test: Creates fixed triplets for testing
    """

    def __init__(self, train_df):
        self.train_df = train_df
        self.arg1 = train_df['s1']
        self.arg2 = train_df['s2']

        
        random_state = np.random.RandomState(29)

        triplets = [[i,
                     random_state.choice(self.label_to_indices[self.test_labels[i].item()]),
                     random_state.choice(self.label_to_indices[
                                             np.random.choice(
                                                 list(self.labels_set - set([self.test_labels[i].item()]))
                                             )
                                         ])
                     ]
                    for i in range(len(self.train_df))]
        self.test_triplets = triplets

    def __getitem__(self, index):
        # Split sentence into words.
        s1_words = self.arg1[idx].split()
        s2_words = self.arg2[idx].split()

        # Add <SOS> and <EOS> tokens.
        s1_words = [self.vocab.sos_token] + s1_words + [self.vocab.eos_token]
        s2_words = [self.vocab.sos_token] + s2_words + [self.vocab.eos_token]

        # Lookup word ids in vocabularies.
        s1_ids = [self.vocab.word2id(word) for word in s1_words]
        s2_ids = [self.vocab.word2id(word) for word in s2_words]
        label = self.label[idx]

        return s1_ids, s2_ids, label

    def __len__(self):
        return len(self.train_df)